In [1]:
from datetime import datetime

In [2]:
#Functions for extracting, logging, and loading
def extract(file_name):
    data = []
    with open(file_name, 'r') as f:
        data = f.readlines()
    return data
def log(message):
    date_format = '%Y-%h-%d-%H:%M:%S'
    now = datetime.now()
    timestamp = now.strftime(date_format)
    with open('log.txt', 'a') as f:
        f.write(message + ': ' + timestamp + '\n')
def load(content, filename):
    with open(filename, 'w') as f:
        for c in content:
            f.write(c + '\n')

In [3]:
#Functions for translating assembly code
jmp_dict = {'':'000', 'JGT':'001', 'JEQ':'010', 'JGE':'011', 'JLT':'100', 'JNE':'101', 'JLE':'110', 'JMP':'111'}
comp_dict = {'0':'0101010', '1':'0111111', '-1':'0111010', 'D':'0001100', 'A':'0110000', 'M':'1110000',
              '!D':'0001101', '!A':'0110001', '!M':'1110001', '-D':'0001111', '-A':'0110011', '-M':'1110011',
              'D+1':'0011111', 'A+1':'0110111', 'M+1':'1110111', 'D-1':'0001110', 'A-1':'0110010', 'M-1':'1110010',
              'D+A':'0000010', 'D+M':'1000010', 'D-A':'0010011', 'D-M':'1010011', 'A-D':'0000111', 'M-D':'1000111',
              'D&A':'0000000', 'D&M':'1000000', 'D|A':'0010101', 'D|M':'1010101'}

def a_instruction(asm_code):
    return '0' + '{0:015b}'.format(int(asm_code[1:]))

def d_instruction(asm_code):
    def jmp(jmp_asm):
        return jmp_dict[jmp_asm]
    
    def dest(dest_asm):
        def dest_helper(find_result):
            if find_result != -1:
                return '1'
            else:
                return '0'
        
        mach_code = ''
        mach_code += dest_helper(dest_asm.find('A'))
        mach_code += dest_helper(dest_asm.find('D'))
        mach_code += dest_helper(dest_asm.find('M'))
        return mach_code
    def comp(comp_asm):
        return comp_dict[comp_asm]
    
    
    line = asm_code
    dest_mach = ''
    comp_mach = ''
    jmp_mach = ''

    eq_ind = line.find('=')
    jmp_ind = line.find(';')
    if eq_ind != -1:
        dest_mach = dest(line[0:eq_ind])
    else:
        dest_mach = '000'
        
    if jmp_ind != -1:
        jmp_mach = jmp(line[jmp_ind+1:])
    else:
        jmp_mach = '000'
            
    if eq_ind != -1 and jmp_ind != -1:
        comp_mach = comp(line[eq_ind+1:jmp_ind])
    elif eq_ind != -1:
        comp_mach = comp(line[eq_ind+1:])
    elif jmp_ind != -1:
        comp_mach = comp(line[:jmp_ind])
            
    mach_code = comp_mach+dest_mach+jmp_mach         
    return mach_code



def label_pass(asm_code):
    def new_label(label, rom_add):
        symbol_dict[label] = str(rom_add)
        return None
    
    label_counter = 0
    out = []
    for i in range(len(asm_code)):
        line = asm_code[i]
        label_ind = line.find('(')
        if label_ind != -1:
            close_ind = line.find(')')
            new_label(line[label_ind+1:close_ind], i-label_counter)
            label_counter += 1
        else:
             out.append(asm_code[i])
    return out       
    
def var_pass(asm_code):
    def replace_var(line):
        return symbol_dict[line]
    def new_var(line):
        global next_mem
        symbol_dict[line] = str(next_mem)
        next_mem += 1
    def var_scan(line):
        if line in symbol_dict:
            return replace_var(line)
        else:
            new_var(line)
            return replace_var(line)
        
    out = []
    for line in asm_code:
        line_c = ''
        a_ind = line.find('@')
        if a_ind != -1 and line[a_ind+1].isalpha():
            line_c += line[a_ind]
            line_c += var_scan(line[a_ind+1:])
        else:
            line_c = line
        out.append(line_c)
    return out
    
            
        
def symbol_converter(asm_code):
    global symbol_dict
    global next_mem
    symbol_dict = {'R0':'0', 'R1':'1', 'R2':'2', 'R3':'3', 'R4':'4', 'R5':'5', 'R6':'6', 'R7':'7', 'R8':'8', 'R9':'9', 
                  'R10':'10', 'R11':'11', 'R12':'12', 'R13':'13', 'R14':'14', 'R15':'15', 'SCREEN':'16384', 'KBD':'24576',
                  'SP':'0', 'LCL':'1', 'ARG':'2', 'THIS':'3', 'THAT': '4'}
    next_mem = 16
    asm_code_fp = label_pass(asm_code)
    asm_code_sp = var_pass(asm_code_fp)

    return asm_code_sp
        
    
def asm_janitor(asm_code):
    out = []
    for line in asm_code:
        remove_ind = 0
        cleaned_line = ''.join(line.split())
        remove_ind = cleaned_line.find('//')
        if remove_ind != -1:
            cleaned_line = cleaned_line[0:remove_ind]
        if cleaned_line:
            out.append(cleaned_line)
    return out  

def asm_translator(asm_code):
    out = []
    asm_code_clean = asm_janitor(asm_code)
    asm_code_L = symbol_converter(asm_code_clean)
    for line in asm_code_L:
        hack_code = ''
        if line[0] == '@':
            hack_code += a_instruction(line)
        else:
            hack_code += '111'
            hack_code += d_instruction(line)
 
        if hack_code:
            out.append(hack_code)
    return out

In [4]:
#Change extension from .asm to .hack
def extension_change(asm_file):
    ext_ind = asm_file.find('.')
    return asm_file[:ext_ind] + '.hack'
#Main function
def ETL(asm_file):
    hack_file = extension_change(asm_file)
    log('Extract ' + asm_file + ' start')
    asm_code = extract(asm_file)
    log('Extract ' + asm_file + ' end')

    log('Translate ' + asm_file + ' start')
    hack_code = asm_translator(asm_code)
    log('Translate ' + asm_file + ' end')

    log('Load ' + hack_file + ' start')
    load(hack_code, hack_file)
    log('Load ' + hack_file + ' end')
    

In [5]:
ETL('add.asm')

In [6]:
ETL('MaxL.asm')

In [7]:
ETL('PongL.asm')

In [8]:
ETL('RectL.asm')

In [9]:
ETL('Rect.asm')

In [10]:
ETL('Pong.asm')

In [11]:
ETL('Max.asm')